<h1 id="basics" style="background:black; padding: 10px 0; border:#FFC300; border-width:5px; border-radius: 10px; border-style:solid; font-family:MV Boli; color:white; line-height: 10px;"> 
    <center> ◽ CONTENTS ◽
    </center>
</h1>

#### [**<span style="color:black">1. Importing the Libraries</span>**](#one)
    
#### [**<span style="color:black">2. Loading the Dataset</span>**](#two)

#### [**<span style="color:black">3. Problem statement</span>**](#three)

#### [**<span style="color:black">4. Exploratory Data Analysis (Phase 1)</span>**](#four)

#### [**<span style="color:black">5. Feature Engineering</span>**](#five)

#### [**<span style="color:black">6. Exploratory Data Analysis (Phase 2)</span>**](#six)

#### [**<span style="color:black">7. Encoding Categorical Features + Train-Test Split</span>**](#seven)

#### [**<span style="color:black">8. Simple Decision Tree Classifier</span>**](#eight)

#### [**<span style="color:black">9. Gradient Boosted Decision Trees using XGBoost</span>**](#nine)

#### [**<span style="color:black">10. Random Forest Classifier</span>**](#ten)

#### [**<span style="color:black">11. Comparison of Models</span>**](#eleven)

#### [**<span style="color:black">12. Insights obtained from EDA</span>**](#twelve)

#### [**<span style="color:black">13. Discussion on Trade Off and Possible Recommendations</span>**](#thirteen)


<a id = "one"></a>
<h1 id="basics" style="background:black; padding: 10px 0; border:#FFC300; border-width:5px; border-radius: 10px; border-style:solid; font-family:MV Boli; color:white; line-height: 10px;"> 
    <center> ◽ IMPORTING LIBRARIES ◽
    </center>
</h1>



In [ ]:
# importing required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display
from itertools import cycle

import category_encoders as ce
import datetime as dt
import warnings

from sklearn import tree
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.model_selection import learning_curve, train_test_split

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


pd.set_option('display.max_columns',None)
warnings.filterwarnings('ignore')


<a id = "two"></a>
<h1 id="basics" style="background:black; padding: 10px 0; border:#FFC300; border-width:5px; border-radius: 10px; border-style:solid; font-family:MV Boli; color:white; line-height: 10px;"> 
    <center> ◽ LOADING THE DATASET ◽
    </center>
</h1>

In [ ]:
df = pd.read_csv('../input/predicting-employee-attrition/train_MpHjUjU.csv')

<a id = "three"></a>
<h1 id="basics" style="background:black; padding: 10px 0; border:#FFC300; border-width:5px; border-radius: 10px; border-style:solid; font-family:MV Boli; color:white; line-height: 10px;"> 
    <center> ◽ PROBLEM STATEMENT ◽
    </center>
</h1>

In [ ]:
df.head()

In [ ]:
print(df.columns)
print("No. of columns : ", len(df.columns))

In [ ]:
df.shape

#### 2. Data types of all the attributes

In [ ]:
df.info()

In [ ]:
df[df.duplicated()]

#### **<span style="color:#483D8B;">Observation: </span>**
 - <strong>So there are no duplicates in our data. </strong>

In [ ]:
#converting categorical features to 'object' type
df['Emp_ID'] = df['Emp_ID'].astype("object")
df['Education_Level'] = df['Education_Level'].astype("object")
df['Gender'] = df['Gender'].astype('object')
df['Joining Designation'] = df['Joining Designation'].astype('object')
df['Designation'] = df['Designation'].astype('object')
df['Quarterly Rating'] = df['Quarterly Rating'].astype('object')

#date-time features
df['MMM-YY'] = pd.to_datetime(df["MMM-YY"]).astype('datetime64[ns]')
df['Dateofjoining'] = pd.to_datetime(df["Dateofjoining"]).astype('datetime64[ns]')
df['LastWorkingDate'] = pd.to_datetime(df["LastWorkingDate"]).astype('datetime64[ns]')

#### 
#### 5. Statistical Summary

In [ ]:
#numerical features
df.describe(include=[np.number]).T

In [ ]:
#categorical features
df.describe(include=['object']).T

In [ ]:
#date-time features
df.describe(exclude = ['object','float64', 'int64'])[0:6].T

#### 
#### 6. Missing Value Detection

In [ ]:
percent = (df.isnull().sum()*100/df.isnull().count()).sort_values(ascending=False)
percent_idx = percent.index
absolute = (df.isnull().sum())[percent_idx]

missing = pd.concat([percent, absolute], axis=1, keys = ['Percentage Missing', 'Total Missing'])
missing[missing["Total Missing"]>0]

#### 7. Unique Values

We will look at the number of unique categories of each categorical feature.

In [ ]:
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
df[cat_cols].nunique()

<a id = "five"></a>
<h1 id="basics" style="background:black; padding: 10px 0; border:#FFC300; border-width:5px; border-radius: 10px; border-style:solid; font-family:MV Boli; color:white; line-height: 10px;"> 
    <center> ◽ Feature Engineering ◽
    </center>
</h1>
We will need to perform certain aggregations so that we are left with one record per employee.

In [ ]:
newdf = pd.DataFrame()

We will now put the unique Employee IDs as a column and aggregate over the values of other columns as necessary.

In [ ]:
newdf["E_ID"] = df["Emp_ID"].unique()

In [ ]:
newdf["Age"] = df.groupby('Emp_ID').agg({'Age':'max'})['Age'].values

In [ ]:
newdf["Gender"] = df.groupby('Emp_ID').agg({'Gender':'last'})['Gender'].values

#### 4. Education Level

In [ ]:
df.groupby("Emp_ID")["Education_Level"].nunique().sort_values().iloc[[0,-1]]

In [ ]:
newdf["Education"] = df.groupby('Emp_ID').agg({'Education_Level':'last'})['Education_Level'].values

#### 5. City

In [ ]:
df.groupby("Emp_ID")["City"].nunique().sort_values().iloc[[0,-1]]

It appears that each employee is only associated to one unique city. So again, we can pick either the first or last.

In [ ]:
newdf["City"] = df.groupby('Emp_ID').agg({'City':'first'})['City'].values

#### 6. Income

In [ ]:
df.groupby("Emp_ID")["Salary"].nunique().sort_values().iloc[[0,-1]]

In [ ]:
newdf['Salary'] = df.groupby('Emp_ID').agg({'Salary':'last'})['Salary'].values

#### 7. Raise (in Salary)

In [ ]:
first_salary = df.groupby('Emp_ID').agg({'Salary':'first'})
last_salary = df.groupby('Emp_ID').agg({'Salary':'last'})

c1 = first_salary > last_salary
c2 = first_salary == last_salary
c3 = first_salary < last_salary

print("Salary has decreased for", c1[c1["Salary"]==True].shape[0], "employees.")
print("Salary has remained unchanged for", c2[c2["Salary"]==True].shape[0], "employees.")
print("Salary has increased for", c3[c3["Salary"]==True].shape[0], "employees.")

In [ ]:
# 1 if income has increased, 0 otherwise.
newdf["Raise"] = np.where(c3["Salary"]==True, 1, 0)

#### 8. Joining Designation and Current Designation (Grade)

In [ ]:
df.groupby("Emp_ID")["Joining Designation"].nunique().sort_values().iloc[[0,-1]]

In [ ]:
df.groupby("Emp_ID")["Designation"].nunique().sort_values().iloc[[0,-1]]

The joining designation of the employees has not changed as expected. The current designation has changed for some (most likely they were promoted).

In [ ]:
newdf["Joining Designation"] = df.groupby('Emp_ID').agg({'Joining Designation':'first'})['Joining Designation'].values
newdf["Current Designation"] = df.groupby('Emp_ID').agg({'Designation':'last'})['Designation'].values

#### 9. Promotion (in Grade)

In [ ]:
first_des = df.groupby('Emp_ID').agg({'Joining Designation':'first'}).rename(columns={'Joining Designation':'Grade'})
last_des = df.groupby('Emp_ID').agg({'Designation':'last'}).rename(columns={'Designation':'Grade'})

c1 = first_des > last_des
c2 = first_des == last_des
c3 = first_des < last_des

print(c1[c1["Grade"]==True].shape[0], "employees were demoted.")
print(c2[c2["Grade"]==True].shape[0], "employees have unchanged grade.")
print(c3[c3["Grade"]==True].shape[0], "employees were promoted.")

In [ ]:
# 1 if promoted, 0 otherwise.
newdf["Promoted"] = np.where(c3["Grade"]==True, 1, 0)

#### 10. Total Business Value

We take the total business value generated by the employee as sum of all such values.

In [ ]:
newdf["Total Business Value"] = df.groupby("Emp_ID").agg({'Total Business Value':'sum'})['Total Business Value'].values

#### 11. Quarterly Rating

We take the latest quarterly rating of the employee.

In [ ]:
newdf["Quarterly Rating"] = df.groupby('Emp_ID').agg({'Quarterly Rating':'last'})['Quarterly Rating'].values

#### 12. Increase in Quarterly Rating

In [ ]:
first_q = df.groupby('Emp_ID').agg({'Quarterly Rating':'first'})
last_q = df.groupby('Emp_ID').agg({'Quarterly Rating':'last'})

c1 = first_q > last_q
c2 = first_q == last_q
c3 = first_q < last_q

print(c1[c1["Quarterly Rating"]==True].shape[0], "employees had a decrease in quarterly rating.")
print(c2[c2["Quarterly Rating"]==True].shape[0], "employees had unchanged quarterly rating.")
print(c3[c3["Quarterly Rating"]==True].shape[0], "employees had an increase in quarterly rating.")

In [ ]:
#we assign -1 for those who had reduced rating
#we assign 0 for unchanged rating
#we assign +1 for those who had increased rating

newdf["qr_increased"] = np.where(c3["Quarterly Rating"]==True, 1, (np.where(c2["Quarterly Rating"]==True, 0, -1)))

#### 13. Experience

In [ ]:
newdf["last_report"] = df.groupby('Emp_ID').agg({'MMM-YY':'last'})['MMM-YY'].values

In [ ]:
newdf["Dateofjoining"] = df.groupby('Emp_ID').agg({'Dateofjoining':'first'})['Dateofjoining'].values

In [ ]:
newdf["Experience"] = newdf["last_report"] - newdf["Dateofjoining"]
newdf["Experience"] = newdf["Experience"].astype("timedelta64[D]")

In [ ]:
newdf.drop(columns=["last_report","Dateofjoining"], inplace=True)

#### 14. Target column

We need to check which employees have left/ attrited.

In [ ]:
newdf["attrited"] = df.groupby("Emp_ID").agg({'LastWorkingDate':'last'})['LastWorkingDate'].values

In [ ]:
# Assign 0 values if last working day does not eist, assign 1 otherwise.
newdf["attrited"] = np.where(newdf["attrited"].isnull(), 0, 1)

<a id = "six"></a>
<h1 id="basics" style="background:black; padding: 10px 0; border:#FFC300; border-width:5px; border-radius: 10px; border-style:solid; font-family:MV Boli; color:white; line-height: 10px;"> 
    <center> ◽ Exploratory Data Analysis (Phase 2) ◽
    </center>
</h1>
We need to repeat certain EDA steps since we now have a new dataset after performing the aggregations.

In [ ]:
newdf.head(3)

In [ ]:
newdf.info()

#### 1. Shape of New Data

In [ ]:
newdf.shape

#### **<span style="color:#483D8B;">Observation: </span>**
 - <strong>So we have 15 columns (of which one is target column) and 2381 observations now.  </strong>



#### 2. Rechecking Missing Values

In [ ]:
percent = (newdf.isnull().sum()*100/newdf.isnull().count()).sort_values(ascending=False)
percent_idx = percent.index
absolute = (newdf.isnull().sum())[percent_idx]

missing = pd.concat([percent, absolute], axis=1, keys = ['Percentage Missing', 'Total Missing'])
missing[missing["Total Missing"]>0]

#### **<span style="color:#483D8B;">Observation: </span>**
 - <strong>So we see that there are no more missing values in our new data after the feature engineering steps.  </strong>

#### 3. Converting features of new data to their respective data type.

We will convert the data types of certain features.

In [ ]:
newdf['E_ID'] = newdf['E_ID'].astype("object")
newdf['Education'] = newdf['Education'].astype("object")
newdf['Gender'] = newdf['Gender'].astype('object')
newdf['Joining Designation'] = newdf['Joining Designation'].astype('object')
newdf['Current Designation'] = newdf['Current Designation'].astype('object')
newdf['Quarterly Rating'] = newdf['Quarterly Rating'].astype('object')
newdf['Raise'] = newdf['Raise'].astype('object')
newdf['Promoted'] = newdf['Promoted'].astype('object')
newdf['qr_increased'] = newdf['qr_increased'].astype('object')
newdf['attrited'] = newdf['attrited'].astype('object')

#### 4. Statistical Summary of New Data

In [ ]:
newdf.describe(include=[np.number]).T

In [ ]:
newdf.describe(include=['object']).T

 - We can drop the unique identifier for the employees as it is no longer necessary.

In [ ]:
newdf.drop(columns=['E_ID'], inplace=True)

#### 5. Unique Values

In [ ]:
cat_cols = newdf.select_dtypes(include=['object']).columns.tolist()
for col in cat_cols:
    print("Unique Values for Column '",col,"' :", newdf[col].unique().tolist())
    print("="*148)
    

We saw above what the unique categories are for each feature. Next we will look at the distribution of attrition across these categorical features.

In [ ]:
for col in cat_cols:
    if col == 'City' or col=='attrited':
        continue
    print(pd.crosstab(newdf[col], newdf["attrited"], normalize='index'))
    print("="*148)

#### 6. Univariate Analysis

#### 6.a. Target Feature

In [ ]:
newdf["attrited"].value_counts(normalize=True)

In [ ]:
sns.set_palette(sns.color_palette("pastel"))
sns.countplot(newdf["attrited"])
plt.show()

#### **<span style="color:#483D8B;">Observation: </span>**
 - <strong>So we see that almost 67.8 % of the employees have left the organization.   </strong>


#### 6.b. Numerical Features

In [ ]:
num_cols = ['Age', 'Salary', 'Total Business Value', 'Experience']


cycol = cycle('cbmr')


fig, ax = plt.subplots(len(num_cols),2, figsize=(16,20))

for i in range(len(num_cols)):
    color_next = next(cycol)
    sns.distplot(newdf[num_cols[i]], ax=ax[i,0], color=color_next)
    ax[i,0].set_title(num_cols[i])
    ax[i,0].set_xlabel('')
    sns.boxplot(newdf[num_cols[i]], width = 0.3, ax=ax[i,1], color=color_next)
    ax[i,1].set_title(num_cols[i]+str("-Boxplot"))
    ax[i,1].set_xlabel('')
    
#plt.suptitle("Univariate Plots for Numerical Columns")
plt.show()

In [ ]:
newdf[num_cols].describe().T

 #### **<span style="color:#483D8B;">Observation: </span>**
 - <strong> The median age of the employees is 33. There are employees as young as 21 years and as old as 58 years.  </strong>
 - <strong> The median income is about 55K. It can range from 10K all the wa to 188K.  </strong>
 - <strong> The median business value generated is 817K. </strong>
 - <strong> The median experience is between 5 to 6 months.   </strong>
 -<strong> It seems that there are outliers. Howver, our data is already small. It might not necessarily be a good idea to drop those outliers. Those may not be because of recording error. </strong>

#### 6.c. Categorical Features

In [ ]:
cat_cols = ['Gender', 'Education', 'Raise', 'Joining Designation', 'Current Designation', 'Promoted', 'Quarterly Rating', 'qr_increased']

fig, ax = plt.subplots(4,2, figsize=(16,14))

for i in range(len(cat_cols)):
    sns.countplot(newdf[cat_cols[i]], ax=ax[i//2,i%2])
    ax[i//2,i%2].set_title(cat_cols[i])
    ax[i//2,i%2].set_xlabel('')
    
plt.show()

- We look at the city column separately since it has many categories.

In [ ]:
for col in cat_cols:
    display(pd.DataFrame(newdf[col].value_counts(normalize=True)))
    


In [ ]:
newdf["City"].value_counts()[0:2]

In [ ]:
plt.figure(figsize=(16,6))
x = newdf['City'].value_counts()[0:30].values
y = newdf['City'].value_counts()[0:30].index.tolist()

sns.barplot(x = x, y = y, orient='h')
plt.title('City')
plt.show()

#### **<span style="color:#483D8B;">Observation: </span>**
 - <strong> So we see that there are 59% male employees and 41% female employees.  </strong>
 - <strong> The percentages of employees with different education levels are almost equal (~33%)  </strong>
 - <strong> 98.2% of the employees did not receive any income increment. Only 1.8% received a raise. </strong>
 - <strong> Almost 43% of the employees joined at lowest designation (1). 34% joined at level 2, 20% at level 3 and below 2% joined at higher levels.   </strong>
 -<strong> Majority (35%) of the employees currently are at designation level 2, followed by designation level 1 (31%) and 3 (26%). Less than 6% of the employees are currently in higher designations. </strong>
 - <strong> Only 17% of the employees received a promotion, while 83% did not. However, we saw that only 1.8% received a raise in income. So it seems like these people are getting ahead in their grade without receiving the financial compensation. This might lead to dissatisfaction. </strong>
  - <strong>Quarterly Rating is lowest (1) for the majority of employees (73%). Very few received rating over 3 (11.5%). </strong>
 - <strong>Quarterly rating increased for only 15% of employees. It reduced for 19% of the employees and remained unchanged for 65%. </strong>
 - <strong> The majority of the employees seem to be associated with city C20.</strong>

#### 7. Bivariate Analysis

#### 7.a. Numerical Columns v/s Attrition

In [ ]:
fig, ax = plt.subplots(4,2, figsize=(16,16))

sns.histplot(newdf[newdf["attrited"]==1]["Age"], ax=ax[0][0], label="Attrited", bins=np.arange(20,60,1), color="pink", alpha=0.5)
sns.histplot(newdf[newdf["attrited"]==0]["Age"], ax=ax[0][0], label="Not Attrited", bins=np.arange(20,60,1), color="red", alpha=0.5)
ax[0][0].legend()

pal1 = {1: "pink", 0: "red"}
sns.boxplot(x=newdf["Age"],y=newdf["attrited"],orient='h', ax=ax[0][1], palette=pal1)

sns.histplot(newdf[newdf["attrited"]==1]["Salary"], ax=ax[1][0], label="Attrited", bins=np.arange(20000,200000,2500), color="skyblue", alpha=0.5)
sns.histplot(newdf[newdf["attrited"]==0]["Salary"], ax=ax[1][0],  label="Not Attrited", bins=np.arange(20000,200000,2500), color="blue", alpha=0.5)
ax[1][0].legend()

pal2 = {1: "skyblue", 0: "blue"}
sns.boxplot(x=newdf["Salary"],y=newdf["attrited"],orient='h', ax=ax[1][1], palette=pal2)

sns.histplot(newdf[newdf["attrited"]==1]["Total Business Value"], ax=ax[2][0], label="Attrited", bins=np.arange(20000,200000,20000), color="lightgreen", alpha=0.5)
sns.histplot(newdf[newdf["attrited"]==0]["Total Business Value"], ax=ax[2][0],  label="Not Attrited", bins=np.arange(20000,200000,20000), color="orange", alpha=0.5)
ax[1][0].legend()

pal3 = {1: "lightgreen", 0: "orange"}
sns.boxplot(x=newdf["Total Business Value"],y=newdf["attrited"],orient='h', ax=ax[2][1], palette=pal3)

sns.histplot(newdf[newdf["attrited"]==1]["Experience"], ax=ax[3][0], label="Attrited", bins=np.arange(0,3000,200), color="pink", alpha=0.5)
sns.histplot(newdf[newdf["attrited"]==0]["Experience"], ax=ax[3][0],  label="Not Attrited", bins=np.arange(0,3000,200), color="magenta", alpha=0.5)
ax[1][0].legend()

pal4 = {1: "pink", 0: "magenta"}
sns.boxplot(x=newdf["Experience"],y=newdf["attrited"],orient='h', ax=ax[3][1], palette=pal4)

plt.show()

In [ ]:
newdf.groupby('attrited')['Age','Salary','Total Business Value', 'Experience'].median()

#### **<span style="color:#483D8B;">Observation: </span>**
 - <strong> The median age for attrited employees is 33, while for non attrited employees it is 34. </strong>
 - <strong> The median income of atrited employees is lower (\~51K) than non attrited employees (\~64K).  </strong>
 - <strong> The median total business value generated by attrited employees (\~46K) is almost 1/6 of the non attrited employees (\~263K) </strong>
 - <strong> The median experience for attrited employees is about 165 days and for non attrited employees it is 186 days.   </strong>

#### 7.b. Categorical Columns vs Attrition

In [ ]:
for col in cat_cols:
    d = pd.DataFrame(newdf.groupby([col, 'attrited'])['Salary'].count()).reset_index().rename(columns={'Salary':'percent'})
    a = d.groupby(col)['percent'].transform('sum')
    d['percent'] = d['percent'].div(a)
    d = d[d['attrited']==1]
    display(d)

In [ ]:
fig, ax = plt.subplots(4,2, figsize=(20,20))

sns.countplot(x = "Gender", data = newdf, hue = "attrited", ax=ax[0][0])
sns.countplot(x = "Education", data = newdf, hue = "attrited", ax=ax[0][1])
sns.countplot(x = "Raise", data = newdf, hue = "attrited", ax=ax[1][0])
sns.countplot(x = "Joining Designation", data = newdf, hue = "attrited", ax=ax[1][1])
sns.countplot(x = "Current Designation", data = newdf, hue = "attrited", ax=ax[2][0])
sns.countplot(x = "Promoted", data = newdf, hue = "attrited", ax=ax[2][1])
sns.countplot(x = "Quarterly Rating", data = newdf, hue = "attrited", ax=ax[3][0])
sns.countplot(x = "qr_increased", data = newdf, hue = "attrited", ax=ax[3][1])
ax[1][1].tick_params(axis='x', rotation=90)
plt.show()

#### **<span style="color:#483D8B;">Observation: </span>**
 - <strong> The attrition rate of male and female employees is similar (\~67-68%).  </strong>
 - <strong> The attrition rate across different education levels is also similar (from 66-69%).  </strong>
 - <strong> The attrition among those who received raise (7%) is far lower than those who did not receive raise (68%). </strong>
 - <strong> The atrition is highest among those whose current designation is 1 (80%) followed by 2 (70%). The rate is around 50-54% for the remaining designation.   </strong>
 -<strong>The attrition rate is 70% for those who were not promoted and 54% for those who were promoted. </strong>
 - <strong> The attrtion rate follows a clear declinig pattern with increasing quarterly rating : 1 (82%), 2 (40%), 3 (16%) and 4 (9%). </strong>
 - <strong>The attrtion rate is highest among those whose quarterly rating decreased (81%), followed by those whose rating remained unchanged (74%) and finally those whose rating increased (23%). </strong>

#### 8. Heatmaps to check Correlation

In [ ]:
sns.heatmap(newdf.corr(), annot=True)
plt.show()

- <strong>There is mopderate correlation between the Total Business Value and Experience features, which is somewhat expected, since the longer a person works, the more likely it is for him to generate positive business value and higher the sum of monthly business values.</strong>

In [ ]:
plt.figure(figsize=(10,8))
df1 = newdf.copy()
cat_cols1 = [
 'Raise',
 'Joining Designation',
 'Current Designation',
 'Promoted',
 'Quarterly Rating',
 'qr_increased',
 'attrited']

for col in cat_cols1:
    df1[col] = df1[col].astype('int64')
    
sns.heatmap(df1.corr(), annot=True, cmap='coolwarm')
plt.show()

#### **<span style="color:#483D8B;">Observation: </span>**
 - <strong> There is moderate correlation between Current Designation and Income which is expected since the higher the designation, the greater the income. </strong>
 - <strong> Also, there is moderate correlation between the current designation and joining designation.</strong>
 - <strong> Again, there is moderate correlation between Experience nd Promotion (the longer a person has been working, the higher the chance of getting promoted. </strong>

In [ ]:
newdf.info()

In [ ]:
newdf.head()

<a id = "seven"></a>
<h1 id="basics" style="background:black; padding: 10px 0; border:#FFC300; border-width:5px; border-radius: 10px; border-style:solid; font-family:MV Boli; color:white; line-height: 10px;"> 
    <center> ◽ Encoding Categorical Features + Train-Test Split ◽
    </center>
</h1>

#### 1. Encoding of Categorical Features

- Raise, Promoted and qr_increased are already represented as 0 or 1.
- Joining Designation, Current Designation and Quarterly Rating could be one hot encoded, however, they have inherent order and it would be wise to preserve that order (say somehow 0 and 4 get grouped together while splitting if we treat it as categorical, which may not make logical sense).
- City could be one hot encoded but there would be too many resultant columns, so we can perform target encoding for it instead.
- Education can be label encoded (0 : College, 1:Bachelor, 2:Master)
- Gender can be represented as 0-1 encoding : Male : 0, Female: 1


In [ ]:
newdf['Education'] = newdf['Education'].map({'College':0,'Bachelor':1,'Master':2})
newdf['Gender'] = newdf['Gender'].map({'Male':0, 'Female':1})

#### 2. Train Test Split

We will first separate out the independent and the dependent columns.

In [ ]:
Y = newdf["attrited"]
Y = Y.astype('int')
X = newdf.drop(columns=['attrited'])

print(X.shape)
print(Y.shape)

Next we perform train test split.

In [ ]:


#X = np.array(X.values.tolist())
#Y = np.array(Y.values.tolist())

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=1001)
print(f"Sizes of the sets created are:\nTraining set:{X_train.shape[0]}\nTest set:{X_test.shape[0]}")

So we see that we have around 1904 training observations and 477 test observations.

#### 3. Target Encoding
Next, we perform target encoding. We fit the encoder on the training data. We use the same encoder to transform our test data (we do not fit on test data so that there is no 'leakage' of target values of the test data).

In [ ]:
import category_encoders as ce

ce_target = ce.TargetEncoder(cols=['City'])

X_train = ce_target.fit_transform(X_train, Y_train)
X_test = ce_target.transform(X_test)

In [ ]:
X_train.head()

Next we shall check the class imbalance of our sets.

In [ ]:
Y_train.value_counts(normalize=True)

In [ ]:
Y_test.value_counts(normalize=True)

#### **<span style="color:#483D8B;">Observation: </span>**
- <strong>So we see that there is similar distribution of attrition in both the train and test data. So we have class imbalance. We have to treat it while we train our models.</strong>

<a id = "eight"></a>
<h1 id="basics" style="background:black; padding: 10px 0; border:#FFC300; border-width:5px; border-radius: 10px; border-style:solid; font-family:MV Boli; color:white; line-height: 10px;"> 
    <center> ◽ Simple Decision Tree Classifier ◽
    </center>
</h1>

### 1. Simple Decision Tree Classifier

 - We will use 5 fold Cross Validation.
 - We will use Stratified K-Fold. Our training Data is only about 1904 observations in size. Of these, we have around 1286 positive class observations and 618 negative class observations, which turns out to be a 68:32 ratio.. If we perform 5 fold stratified CV on this, we should roughly get 257 positive and 124 negative points. Since the size is small, if we choose the folds randomly, we might end up with different ratio than expected, in each of the folds.
 - We will use GridSearch to tune the hyperparameters, since we are training a single decision tree on a small data and it will not take a lot of time.
 - We will treat class imbalance in the following ways:
     - Using class weights : We will keep this as a hyperparameter
     - Using oversampling (by SMOTE)
     - We will NOT use undersampling since the data size is already small.
 - We will consider maximum tree depth, and maximum number of leaf nodes as the other hyperparameters.
 

#### 1.a. Treating imbalance using Class Weights

In [ ]:
cl1 = Y_train.value_counts().values[0]
cl0 = Y_train.value_counts().values[1]
r0 = 1
r1 = round(cl1/cl0,2)
print("Ratio of class0 to class1 is", r0, ":", r1)

In [ ]:

#We could also treat weights as hyperparameters.
weights = {0:r1, 1:r0}
model1 = DecisionTreeClassifier()

#hyper parameters which we will tune using GridSearch
params = {
    "class_weight" : [{0:1, 1:1}, {0:r1, 1:r0}, {0:3, 1:1},],
    "max_depth" : [5, 7, 10], #the maximum depth of the tree
    "max_leaf_nodes" : [20, 25, 30] #the number of leaf nodes of the tree
}

#stratified k fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 1001)
#specifying our classifier
clf = GridSearchCV(model1, params, scoring = "f1", cv=skf.split(X_train,Y_train))
#Training the model
clf.fit(X_train, Y_train)

In [ ]:
#The classifier with the best hyper parameters
print(clf.best_estimator_)

#### **<span style="color:#483D8B;">Observation: </span>**
- <strong>It turns out that the original class weighting still gives us better performance. </strong>

#### Learning Curves
Next we will look at the learning curve. It plots the 'Score' obtained from various models trained on different sizes of the training data. 
Please look at https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html for details on other possible plots.

In [ ]:
# Learning Curves


def plot_learning_curve(estimator, X, Y, title):

    train_sizes, train_scores, test_scores, _, _ = learning_curve(estimator,X,Y,return_times=True)

    fig, axes = plt.subplots(1, 1, figsize = (10, 5))

    axes.set_title(title)
    axes.plot
    axes.set_xlabel("Training examples")
    axes.set_ylabel("Score")


    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    # Plot learning curve
    axes.grid()
    axes.fill_between(
      train_sizes,
      train_scores_mean - train_scores_std,
      train_scores_mean + train_scores_std,
      alpha=0.1,
      color="r",
    )
    axes.fill_between(
      train_sizes,
      test_scores_mean - test_scores_std,
      test_scores_mean + test_scores_std,
      alpha=0.1,
      color="g",
    )
    axes.plot(
      train_sizes, train_scores_mean, "o-", color="r", label="Training score"
      )
    axes.plot(
      train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
      )
    axes.legend(loc="best")

    plt.show()

In [ ]:
model1 = clf.best_estimator_

model1.fit(X_train, Y_train)

plot_learning_curve(model1, X_train, Y_train, "Decision Trees")

print(model1.score(X_train, Y_train))

#### **<span style="color:#483D8B;">Observation: </span>**
 - So we see that the training score is very high at the beginning and decreases and the cross-validation score is low at the beginning and increases.

- In the beginning, with very few samples our model is overfitting. As the number of samples increases, our model starts generalizing better and our CV score improves. 

- We see that our training and CV scores appear to be close at 1900 samples. We might be able to get only a slight improvement with more training samples, but not too much.

- The highest cross validation score obtained was around 0.84.

#### Visualizing what the tree looks like
Next, we could also visualize the tree that we have trained.

In [ ]:


plt.figure(figsize=(24,12))  # set plot size (denoted in inches)
tree.plot_tree(model1, fontsize=10)
plt.show()

#### Test Performance

In [ ]:
print(f"Training F1 score:{model1.score(X_train, Y_train)}\nTest F1 score: {model1.score(X_test, Y_test)}")

#### 1.b. Treating Class Imbalance using SMOTE Oversampling

In [ ]:
smote = SMOTE(random_state=1001)
X_train_ov, Y_train_ov = smote.fit_resample(X_train, Y_train)

In [ ]:
Y_train_ov.value_counts() #We now have equal observations in both classes,

In [ ]:
#stratified k fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 1001)

model1_ov = DecisionTreeClassifier()

#parameters which we will tune using GridSearch. We do not need class weights as we have balanced our data.
params_ov = {
    "max_depth" : [3, 5, 7], #the maximum depth of the tree
    "max_leaf_nodes" : [15, 20, 25] #the number of leaf nodes of the tree
}

#specifying our classifier
clf_ov = GridSearchCV(model1_ov, params_ov, scoring = "f1", cv=skf.split(X_train,Y_train))
#Training the model
clf_ov.fit(X_train_ov, Y_train_ov)

#### Learning Curve

In [ ]:
model1_ov = clf_ov.best_estimator_

model1_ov.fit(X_train_ov, Y_train_ov)

plot_learning_curve(model1_ov, X_train_ov, Y_train_ov, "Decision Trees - with SMOTE Oversampling")

print(model1_ov.score(X_train_ov, Y_train_ov))

#### **<span style="color:#483D8B;">Observation: </span>**
- We obtain a similar learning curve as before, but looks like our CV score is still increasing but only a little. The highest cross validation score obtained was around 0.83 - 0.84.

#### Test Performace

In [ ]:
print(f"Training F1 score:{model1_ov.score(X_train_ov, Y_train_ov)}\nTest F1 score: {model1_ov.score(X_test, Y_test)}")

### Comparing Test Performance and Feature Importance of the two Decision Tree Classifiers

#### 1. Feature Importance

In [ ]:
dt_feat_imp = model1.feature_importances_
dt_feat_imp_ov = model1_ov.feature_importances_

fig, ax = plt.subplots(1,2, figsize=(22, 5))

sns.barplot(y = list(X_train.columns), x = dt_feat_imp, orient='h', ax=ax[0])
ax[0].set_title("Feature Importances - DT")
sns.barplot(y = list(X_train.columns), x = dt_feat_imp_ov, orient='h', ax=ax[1])
ax[1].set_title("Feature Importances - DT with Oversampling")
plt.show()

#### 2. Precision, Recall and F1 scores

We already saw that the F1 score on test data was higher for the base model without oversampling.
We will check the classification reports for the two models.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve, auc
Y_test_pred = model1.predict(X_test)
Y_test_pred_ov = model1_ov.predict(X_test)
print("DT model:")
print(classification_report(Y_test, Y_test_pred))
print("-"*60)
print("\nDT model with oversampling :")
print(classification_report(Y_test, Y_test_pred_ov))

#### **<span style="color:#483D8B;">Observation: </span>**
 - We look at the precision, recall and F1 scores for the '1' (attrited) class, since that is what we are trying to predict.

- So the base model has Precision, Recall and F1 score of 0.81, 0.93 and 0.86 respectively.
- And the model with oversampling has the values as 0.84, 0.83 and 0.83 respectively.

- <strong>So the precision is higher for the second model, but the recall and F1 score are better for the base model which used class weights as hyper parameters. </strong>

#### 3. Accuracy and Confusion Matrices

In [ ]:
print("Test accuracy for simple DT -", accuracy_score(Y_test, Y_test_pred))
print("Test accuracy for DT with oversampling -", accuracy_score(Y_test, Y_test_pred_ov))

In [ ]:
print("Confusion matrix format:")
print("   0   1")
print("   _____")
print("0 |__|__|")
print("1 |__|__|")
print("-"*60)

print("\nDT model:")
print(confusion_matrix(Y_test, Y_test_pred))
print("-"*60)
print("\nDT model with oversampling :")
print(confusion_matrix(Y_test, Y_test_pred_ov))

#### **<span style="color:#483D8B;">Observation: </span>**
- <strong>So once again we see that the base DT seems to be performng better w.r.t prediction of positive class and also has better accuracy.</strong>

#### 4. ROC Curves, Area under ROC Curves

In [ ]:
probs_y=model1.predict_proba(X_test) 
precision, recall, thresholds = precision_recall_curve(Y_test, probs_y[:, 1]) 
probs_y_ov=model1_ov.predict_proba(X_test) 
precision_ov, recall_ov, thresholds_ov = precision_recall_curve(Y_test, probs_y_ov[:, 1]) 


fpr, tpr, thresholds = roc_curve(Y_test, probs_y[:,1], drop_intermediate=False)
roc_auc = auc(fpr, tpr)
fpr_ov, tpr_ov, thresholds_ov = roc_curve(Y_test, probs_y_ov[:,1], drop_intermediate=False)
roc_auc_ov = auc(fpr_ov, tpr_ov)

fig, ax = plt.subplots(1,2, figsize=(16,4))

plt.suptitle('Receiver Operating Characteristics')

ax[0].plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
ax[0].legend(loc = 'lower right')
ax[0].plot([0, 1], [0, 1],'r--')
ax[0].set_xlim([0, 1])
ax[0].set_ylim([0, 1])
ax[0].set_ylabel('True Positive Rate')
ax[0].set_xlabel('False Positive Rate')
ax[0].set_title("DT")


ax[1].plot(fpr_ov, tpr_ov, 'g', label = 'AUC = %0.2f' % roc_auc_ov)
ax[1].legend(loc = 'lower right')
ax[1].plot([0, 1], [0, 1],'r--')
ax[1].set_xlim([0, 1])
ax[1].set_ylim([0, 1])
ax[1].set_ylabel('True Positive Rate')
ax[1].set_xlabel('False Positive Rate')
ax[1].set_title("DT with oversampling")


plt.plot()

#### **<span style="color:#483D8B;">Observation: </span>**
- <strong>So we see that the base DT model and the oversampled data model have same AUC of 0.83</strong>

<strong>So from <mark>Section 1</mark>, the best model seems to be : <mark>model1</mark></strong>
#### 

<a id = "nine"></a>
<h1 id="basics" style="background:black; padding: 10px 0; border:#FFC300; border-width:5px; border-radius: 10px; border-style:solid; font-family:MV Boli; color:white; line-height: 10px;"> 
    <center> ◽ Gradient Boosted Decision Trees using XGBoost ◽
    </center>
</h1>

### 2. Gradient Boosted Decision Trees (XgBoost)

 - We will again use 5 fold Cross Validation.
 - We will use Stratified K-Fold.
 - We will use GridSearch and try out RandomSearch to tune the hyperparameters. If GridSearch ends up taking a lot of time to  train, we might go with RandomSearch. If it manages to finish within reasonable time, we can stick with GridSearch.
 - We will treat class imbalance in the following ways:
     - Using class weights : We will keep this as a hyperparameter
     - Using oversampling (by SMOTE)
 - We will consider maximum tree depth, learning rate, row sampling and column sampling rates as the other hyperparameters.
 - We will use 100 base learners.

#### 2.a. Using Class Weights

- According to the XGBoost documentation, a typical value to consider to balance the positive and negative weights is:

sum(negative instances) / sum(positive instances)

- scale_pos_weight = total_negative_examples / total_positive_examples

In [ ]:
Y_train.value_counts()

#### Random Search
We will first look at Random Search.

In [ ]:


for col in X_train.columns:
    X_train[col] = X_train[col].astype('int')
    X_test[col] = X_test[col].astype('int')

Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')


scale_pos_w = 606/1298
#hyper-parameters
params = {
        'scale_pos_weight' : [0.5, 1, scale_pos_w, 5],
        'learning_rate': [0.1, 0.5, 0.8],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

#XgBoost Classifier
xgb = XGBClassifier(n_estimators=100, objective='binary:logistic')

folds = 5
#Stratified k-folds
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state = 1001)

#model with random search for hyper parameter tuning
random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=50, scoring='f1', n_jobs=4, cv=skf.split(X_train,Y_train), verbose=0, random_state=1001)

#Training
start = dt.datetime.now()
random_search.fit(X_train, Y_train)
end = dt.datetime.now()

#### Best Model obtained by Random Search

In [ ]:
print('\n Best hyperparameters:')
print(random_search.best_params_)
random_best = random_search.best_params_

#### **<span style="color:#483D8B;">Observation: </span>**
 - So just like the simple DT, we once again end up obtaining best results with original class weights.

In [ ]:
subsample = random_best['subsample']
max_depth = random_best['max_depth']
learning_rate = random_best['learning_rate']
colsample_bytree = random_best['colsample_bytree']
scale_pos_weight = random_best['scale_pos_weight']

best_xgb_random = XGBClassifier(n_estimators=100, objective='binary:logistic', subsample=subsample, max_depth=max_depth, learning_rate=learning_rate, colsample_bytree=colsample_bytree, scale_pos_weight=scale_pos_weight)
best_xgb_random.fit(X_train, Y_train)

#### Learning Curve

In [ ]:
plot_learning_curve(best_xgb_random, X_train, Y_train, "XgBoost with Random Search")

#### **<span style="color:#483D8B;">Observation: </span>**
- The highest cross validation score obtained was around 0.85. It might improve with more training samples.

#### Test Performance

In [ ]:
print(f"Time taken for training : {end - start}\nTraining F1 score:{best_xgb_random.score(X_train, Y_train)}\nTest F1 score: {best_xgb_random.score(X_test, Y_test)}")

#### Grid Search

In [ ]:
#hyper parameters for grid search
params = {
        'scale_pos_weight' : [0.5, 1, scale_pos_w, 5],
        'learning_rate': [0.1, 0.5, 0.8],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

#XgBoost Classifier
xgb = XGBClassifier(n_estimators=100, objective='binary:logistic')

folds = 5
#Stratified k-folds
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state = 1001)

#model with grid search for hyperparameter tuning
grid_search = GridSearchCV(xgb, params, scoring='f1', n_jobs=4, cv=skf.split(X_train,Y_train))


start = dt.datetime.now()
grid_search.fit(X_train, Y_train)
end = dt.datetime.now()

#### Best model obtained by Grid Search

In [ ]:
print('\n Best hyperparameters:')
print(grid_search.best_params_)
grid_best = grid_search.best_params_

In [ ]:
subsample = grid_best['subsample']
max_depth = grid_best['max_depth']
learning_rate = grid_best['learning_rate']
colsample_bytree = grid_best['colsample_bytree']
scale_pos_weight = random_best['scale_pos_weight']

best_xgb_grid = XGBClassifier(n_estimators=100, objective='binary:logistic', subsample=subsample, max_depth=max_depth, learning_rate=learning_rate, colsample_bytree=colsample_bytree, scale_pos_weight=scale_pos_weight)
best_xgb_grid.fit(X_train, Y_train)

#### Learning Curve

In [ ]:
plot_learning_curve(best_xgb_grid, X_train, Y_train, "XgBoost with Grid Search")

#### **<span style="color:#483D8B;">Observation: </span>**
- The highest cross validation score obtained was slightly above 0.85.

#### Test Performance

In [ ]:
print(f"Time taken for training : {end - start}\nTraining F1 score:{best_xgb_grid.score(X_train, Y_train)}\nTest F1 score: {best_xgb_grid.score(X_test, Y_test)}")

#### **<span style="color:#483D8B;">Observation: </span>**
- <strong>So, it appears that the performance of the model obtained by random search, on test data, is better than that obtained by grid search (even though the best CV scores for Grid Search model is only slightly higher than that of the Random Search Model.</strong>

<strong>We will pick the <mark>random search model</mark> from section 2.a. for our later comparisons. </strong>

#### 2.b. Model on Oversampled Data

#### Random Search

In [ ]:
for col in X_train_ov.columns:
    X_train_ov[col] = X_train_ov[col].astype('int')
Y_train_ov = Y_train_ov.astype('int')

#hyper-parameters
params = {
        'learning_rate': [0.1, 0.5, 0.8],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

#XgBoost Classifier
xgb = XGBClassifier(n_estimators=100, objective='binary:logistic')

folds = 5
#Stratified k-folds
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state = 1001)

#model with random search for hyper parameter tuning
random_search_ov = RandomizedSearchCV(xgb, param_distributions=params, n_iter=50, scoring='f1', n_jobs=4, cv=skf.split(X_train_ov,Y_train_ov), verbose=0, random_state=1001 )

#Training
start = dt.datetime.now()
random_search_ov.fit(X_train_ov, Y_train_ov)
end = dt.datetime.now()

#### Best Model obtained by Random Search

In [ ]:
print('\n Best hyperparameters:')
print(random_search_ov.best_params_)
random_best_ov = random_search_ov.best_params_

In [ ]:
subsample = random_best_ov['subsample']
max_depth = random_best_ov['max_depth']
learning_rate = random_best_ov['learning_rate']
colsample_bytree = random_best_ov['colsample_bytree']

best_xgb_random_ov = XGBClassifier(n_estimators=100, objective='binary:logistic', subsample=subsample, max_depth=max_depth, learning_rate=learning_rate, colsample_bytree=colsample_bytree)
best_xgb_random_ov.fit(X_train_ov, Y_train_ov)

In [ ]:
print(f"Time taken for training : {end - start}\nTraining F1 score:{best_xgb_random_ov.score(X_train_ov, Y_train_ov)}\nTest F1 score: {best_xgb_random_ov.score(X_test, Y_test)}")

#### Grid Search

In [ ]:
#hyper parameters for grid search
params = {
        'learning_rate': [0.1, 0.5, 0.8],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

#XgBoost Classifier
xgb = XGBClassifier(n_estimators=100, objective='binary:logistic')

#Stratified k-folds
folds = 5
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state = 1001)

#model with grid search for hyperparameter tuning
grid_search_ov = GridSearchCV(xgb, params, scoring='f1', n_jobs=4, cv=skf.split(X_train_ov,Y_train_ov))


start = dt.datetime.now()
grid_search_ov.fit(X_train_ov, Y_train_ov)
end = dt.datetime.now()

In [ ]:
print('\n Best hyperparameters:')
print(grid_search_ov.best_params_)
grid_best_ov = grid_search_ov.best_params_

- <strong>So we see that both Random and Grid Search gives us the same model.</strong>

### Comparing Test Performance and Feature Importance of the two XgBoost Ensembles

- So we pick the random search model from 2.a <mark>(best_xgb_random)</mark> and the model from 2.b <mark>(best_xgb_random_ov)</mark> for our comparison.

#### Feature Importances

In [ ]:
xgb_feat_imp = best_xgb_random.feature_importances_
xgb_feat_imp_ov = best_xgb_random_ov.feature_importances_

fig, ax = plt.subplots(1,2, figsize=(22, 5))

sns.barplot(y = list(X_train.columns), x = xgb_feat_imp, orient='h', ax=ax[0])
ax[0].set_title("Feature Importances - XGBoost")
sns.barplot(y = list(X_train.columns), x = xgb_feat_imp_ov, orient='h', ax=ax[1])
ax[1].set_title("Feature Importances - XGBoost with Oversampling")
plt.show()

#### Precision, Recall and F1 scores

We already saw that the F1 score on test data was higher for the base model without oversampling.
We will check the classification reports for the two models.

In [ ]:
Y_test_pred = best_xgb_random.predict(X_test)
Y_test_pred_ov = best_xgb_random_ov.predict(X_test)
print("XgBoost model:")
print(classification_report(Y_test, Y_test_pred))
print("-"*60)
print("\nXgBoost model with oversampling :")
print(classification_report(Y_test, Y_test_pred_ov))

#### **<span style="color:#483D8B;">Observation: </span>**
- <strong>We look at the precision, recall and F1 scores for the '1' (attrited) class, since that is what we are trying to predict.</strong>

- <strong>So the base model has Precision, Recall and F1 score of 0.84, 0.91 and 0.87 respectively.</strong>
- <strong>And the model with oversampling has the values as 0.85, 0.85 and 0.85 respectively.</strong>

- <strong>So the precision is higher for the second model, but the recall and F1 score are better for the base xgboost model which used class weights as hyper parameters.</strong>

#### Accuracy and Confusion Matrices

In [ ]:
print("Test accuracy for XGBoost -", accuracy_score(Y_test, Y_test_pred))
print("Test accuracy for XGBoost with oversampling -", accuracy_score(Y_test, Y_test_pred_ov))

In [ ]:
print("Confusion matrix format:")
print("   0   1")
print("   _____")
print("0 |__|__|")
print("1 |__|__|")
print("-"*60)

print("\nXGBoost model:")
print(confusion_matrix(Y_test, Y_test_pred))
print("-"*60)
print("\nXGBoost model with oversampling :")
print(confusion_matrix(Y_test, Y_test_pred_ov))

#### **<span style="color:#483D8B;">Observation: </span>**
- <strong>So once again we see that the base XGBoost seems to be performng better w.r.t prediction of positive class and also has better accuracy.</strong>

#### ROC Curves, Area under ROC Curves

In [ ]:
probs_y=best_xgb_random.predict_proba(X_test) 
precision, recall, thresholds = precision_recall_curve(Y_test, probs_y[:, 1]) 
probs_y_ov=best_xgb_random_ov.predict_proba(X_test) 
precision_ov, recall_ov, thresholds_ov = precision_recall_curve(Y_test, probs_y_ov[:, 1]) 


fpr, tpr, thresholds = roc_curve(Y_test, probs_y[:,1], drop_intermediate=False)
roc_auc = auc(fpr, tpr)
fpr_ov, tpr_ov, thresholds_ov = roc_curve(Y_test, probs_y_ov[:,1], drop_intermediate=False)
roc_auc_ov = auc(fpr_ov, tpr_ov)

fig, ax = plt.subplots(1,2, figsize=(16,4))

plt.suptitle('Receiver Operating Characteristics')

ax[0].plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
ax[0].legend(loc = 'lower right')
ax[0].plot([0, 1], [0, 1],'r--')
ax[0].set_xlim([0, 1])
ax[0].set_ylim([0, 1])
ax[0].set_ylabel('True Positive Rate')
ax[0].set_xlabel('False Positive Rate')
ax[0].set_title("XgBoost")

ax[1].plot(fpr_ov, tpr_ov, 'g', label = 'AUC = %0.2f' % roc_auc_ov)
ax[1].legend(loc = 'lower right')
ax[1].plot([0, 1], [0, 1],'r--')
ax[1].set_xlim([0, 1])
ax[1].set_ylim([0, 1])
ax[1].set_ylabel('True Positive Rate')
ax[1].set_xlabel('False Positive Rate')
ax[1].set_title("XgBoost with oversampling")

plt.plot()

#### **<span style="color:#483D8B;">Observation: </span>**
- <strong> So the AUC of the original XGBoost model is better than the AUC of the model trained on oversampled data.</strong>

<strong>So from Section 2, the best model seems to be : <mark>best_xgb_random</mark> </strong>
#### 

<a id = "ten"></a>
<h1 id="basics" style="background:black; padding: 10px 0; border:#FFC300; border-width:5px; border-radius: 10px; border-style:solid; font-family:MV Boli; color:white; line-height: 10px;"> 
    <center> ◽ Random Forest Classifier ◽
    </center>
</h1>

#### 
### 3. Random Forest 

 - We will again use 5 fold Cross Validation.
 - We will use Stratified K-Fold.
 - We will use GridSearch to tune the hyperparameters.
 - We will treat class imbalance by treating class weights as hyperparameters.
 - We will consider maximum tree depth, row sampling and number of parallel trees as the other hyperparameters.

In [ ]:


class_wts = [{ 0:1, 1:scale_pos_w },'balanced']
#hyper parameters for grid search
params = {
        'class_weight': class_wts,
        'max_samples': [0.6, 0.8],
        'max_depth': [5, 10, 15],
        'n_estimators':[300, 500]
        }

#RF classifier
rf = RandomForestClassifier(random_state=1001)

folds = 5
#Stratified k-folds
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state = 1001)

#model with grid search for hyperparameter tuning
grid_search_rf = GridSearchCV(rf, params, scoring='f1', n_jobs=4, cv=skf.split(X_train,Y_train))

start = dt.datetime.now()
grid_search_rf.fit(X_train, Y_train)
end = dt.datetime.now()

#### Best Random Forest Model obtained using Grid Search

In [ ]:
print('\n Best hyperparameters:')
print(grid_search_rf.best_params_)
grid_best_rf = grid_search_rf.best_params_

In [ ]:
max_samples = grid_best_rf['max_samples']
max_depth = grid_best_rf['max_depth']
n_estimators = grid_best_rf['n_estimators']
class_weight = grid_best_rf['class_weight']

best_rf = RandomForestClassifier(random_state=1001, max_samples=max_samples, max_depth=max_depth, 
                                n_estimators=n_estimators, class_weight=class_weight)
best_rf.fit(X_train, Y_train)

In [ ]:
print(f"Time taken for training : {end - start}\nTraining F1 score:{best_rf.score(X_train, Y_train)}\nTest F1 score: {best_rf.score(X_test, Y_test)}")

#### **<span style="color:#483D8B;">Observation: </span>**
- <strong>So our random forest model seems to be performing worse than the XGBoost model in terms of the F1 score of the attrited class, on test data. </strong>

<a id = "eleven"></a>
<h1 id="basics" style="background:black; padding: 10px 0; border:#FFC300; border-width:5px; border-radius: 10px; border-style:solid; font-family:MV Boli; color:white; line-height: 10px;"> 
    <center> ◽ Comparison of Different Models ◽
    </center>
</h1>

#### 

### Comparing the simple decision tree, random forest and gradient boosted decision trees models.

#### 1. Feature Importances

In [ ]:
dt_feat_imp = model1.feature_importances_
xgb_feat_imp = best_xgb_random.feature_importances_
rf_feat_imp = best_rf.feature_importances_

df_dt = pd.DataFrame()
df_dt['data'] = dt_feat_imp
df_dt['mdl'] = ['Decision Tree' for _ in range(len(dt_feat_imp))]
df_xgb = pd.DataFrame()
df_xgb['data'] = xgb_feat_imp
df_xgb['mdl'] = ['XGBoost' for _ in range(len(xgb_feat_imp))]
df_rf = pd.DataFrame()
df_rf['data'] = rf_feat_imp
df_rf['mdl'] = ['Random Forest' for _ in range(len(rf_feat_imp))]
final = df_dt.append(df_xgb.append(df_rf))
final["col"] = list(X_train.columns)*3

plt.figure(figsize=(12,12))
sns.barplot(y=final['col'], x=final['data'], hue=final['mdl'], orient='h')
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.show()

#### **<span style="color:#483D8B;">Observation: </span>**
- We can see that the feature 'Quarterly Rating' seems to be very important for the Decision Tree and XGBoost models.

- For Random Forest, Experience seems to be the most important feature, followed by Quarterly Rating. Experience is also important for Decision Tree Model. But it's importance is quite low for the XGBoost model. We observe something similar for the Total Business Value feature.

- The features Age and Income are important for Random Forest, but not so much for Decision Tree or XGBoost.

#### 
#### 2. Precision, Recall and F1 scores

In [ ]:
Y_test_pred_dt = model1.predict(X_test)
Y_test_pred_xgb = best_xgb_random.predict(X_test)
Y_test_pred_rf = best_rf.predict(X_test)

print("Decision Tree model:")
print(classification_report(Y_test, Y_test_pred_dt))
print("-"*60)
print("\nXgBoost model:")
print(classification_report(Y_test, Y_test_pred_xgb))
print("-"*60)
print("\nRandom Forest:")
print(classification_report(Y_test, Y_test_pred_rf))

#### **<span style="color:#483D8B;">Observation: </span>**
- <strong>The <mark>XGBoost</mark> model has the highest <mark>precision</mark>. It's recall is one percent lower than the Decision Tree Model. However, we get the best F1 score for the XGBoost model. </strong>
- <strong>The <mark>F1 score</mark> for <mark>XGBoost model</mark> is <mark>0.87</mark>, while for the other two models it is 0.86.</strong>

- <strong>If we look at <mark>weighted F1 score</mark>, then <mark>XGBoost</mark> appears to be the clear winner with a score of <mark>0.82</mark>, whereas the score for the Decision Tree model is 0.8 and for the Random Forest model, is 0.81.</strong>

#### 
#### 3. Accuracy and Confusion Matrices

In [ ]:
print("Test accuracy for Decision Tree -", accuracy_score(Y_test, Y_test_pred_dt))
print("Test accuracy for XGBoost -", accuracy_score(Y_test, Y_test_pred_xgb))
print("Test accuracy for Random Forest -", accuracy_score(Y_test, Y_test_pred_rf))

In [ ]:
print("Confusion matrix format:")
print("   0   1")
print("   _____")
print("0 |__|__|")
print("1 |__|__|")
print("-"*60)

print("\nDecision Tree:")
print(confusion_matrix(Y_test, Y_test_pred_dt))
print("-"*60)
print("\nXGBoost model:")
print(confusion_matrix(Y_test, Y_test_pred_xgb))
print("-"*60)
print("\nRandom Forest:")
print(confusion_matrix(Y_test, Y_test_pred_rf))

#### **<span style="color:#483D8B;">Observation: </span>**
- <strong>In terms of accuracy again, the <mark>XGBoost</mark> model has highest score of <mark>0.82</mark>, while the other models have around 0.80.</strong>

- <strong>If we look at the confusion matrix, again the <mark>XGBoost</mark> model seems to be performing <mark>reasonably well</mark> in terms of reducing misclassifications of both the positive and negative classes. </strong>

- <strong>Decision tree achieves only marginally better performance for positive class, but misclassifies a lot of negative class observations. Random forest does better on negative class than decision tree, but make it worse for the positive class. So the <mark>XGBoost</mark> model provides a balance between the two. </strong>

#### 
#### 4. ROC Curves, Area under ROC Curves

In [ ]:
probs_y_dt=model1.predict_proba(X_test) 
precision_dt, recall_dt, thresholds_dt = precision_recall_curve(Y_test, probs_y_dt[:, 1]) 
probs_y_xgb=best_xgb_random.predict_proba(X_test) 
precision_xgb, recall_xgb, thresholds_xgb = precision_recall_curve(Y_test, probs_y_xgb[:, 1]) 
probs_y_rf = best_rf.predict_proba(X_test)
precision_rf, recall_rf, thresholds_rf = precision_recall_curve(Y_test, probs_y_rf[:, 1]) 


fpr_dt, tpr_dt, thresholds_dt = roc_curve(Y_test, probs_y_dt[:,1], drop_intermediate=False)
roc_auc_dt = auc(fpr_dt, tpr_dt)
fpr_xgb, tpr_xgb, thresholds_xgb = roc_curve(Y_test, probs_y_xgb[:,1], drop_intermediate=False)
roc_auc_xgb = auc(fpr_xgb, tpr_xgb)
fpr_rf, tpr_rf, thresholds_rf = roc_curve(Y_test, probs_y_rf[:,1], drop_intermediate=False)
roc_auc_rf = auc(fpr_rf, tpr_rf)

fig, ax = plt.subplots(1,3, figsize=(18,4))

plt.suptitle('Receiver Operating Characteristics')

ax[0].plot(fpr_dt, tpr_dt, 'b', label = 'AUC = %0.2f' % roc_auc_dt)
ax[0].legend(loc = 'lower right')
ax[0].plot([0, 1], [0, 1],'r--')
ax[0].set_xlim([0, 1])
ax[0].set_ylim([0, 1])
ax[0].set_ylabel('True Positive Rate')
ax[0].set_xlabel('False Positive Rate')
ax[0].set_title("Decision Tree")

ax[1].plot(fpr_xgb, tpr_xgb, 'g', label = 'AUC = %0.2f' % roc_auc_xgb)
ax[1].legend(loc = 'lower right')
ax[1].plot([0, 1], [0, 1],'r--')
ax[1].set_xlim([0, 1])
ax[1].set_ylim([0, 1])
ax[1].set_ylabel('True Positive Rate')
ax[1].set_xlabel('False Positive Rate')
ax[1].set_title("XgBoost")

ax[2].plot(fpr_rf, tpr_rf, 'orange', label = 'AUC = %0.2f' % roc_auc_rf)
ax[2].legend(loc = 'lower right')
ax[2].plot([0, 1], [0, 1],'r--')
ax[2].set_xlim([0, 1])
ax[2].set_ylim([0, 1])
ax[2].set_ylabel('True Positive Rate')
ax[2].set_xlabel('False Positive Rate')
ax[2].set_title("Random Forest")

plt.plot()

#### **<span style="color:#483D8B;">Observation: </span>**
- The AUC for both XGBoost and Random Forests seems to be same (0.88) while for the decision tree it is 0.82.

<a id = "twelve"></a>
<h1 id="basics" style="background:black; padding: 10px 0; border:#FFC300; border-width:5px; border-radius: 10px; border-style:solid; font-family:MV Boli; color:white; line-height: 10px;"> 
    <center> ◽ Insights obtained using Exploratory Data Analysis ◽
    </center>
</h1>

#### 
### Insights from EDA

- Almost 67.8 % of the employees have left the organization. 
- The median age of the employees is 33. There are employees as young as 21 years and as old as 58 years.
- The median income is about 55K. It can range from 10K all the way to 188K.
- The median business value generated is 817K.
- The median experience is between 5 to 6 months. 
- We see that there are 59% male employees and 41% female employees.
- The percentages of employees with different education levels are almost equal (~33%).
- 98.2% of the employees did not receive any income increment. Only 1.8% received a raise.
- Almost 43% of the employees joined at lowest designation (1). 34% joined at level 2, 20% at level 3 and below 2% joined at higher levels.
- Majority (35%) of the employees currently are at designation level 2, followed by designation level 1 (31%) and 3 (26%). Less than 6% of the employees are currently in higher designations.
- Only 17% of the employees received a promotion, while 83% did not. However, we saw that only 1.8% received a raise in income. So it seems like these people are getting ahead in their grade without receiving the financial compensation. This might lead to dissatisfaction. 
- Quarterly Rating is lowest (1) for the majority of employees (73%). Very few received rating over 3 (11.5%).
- Quarterly rating increased for only 15% of employees. It reduced for 19% of the employees and remained unchanged for 65%.
- The majority of the employees seem to be associated with city C20.
- The median age for attrited employees is 33, while for non attrited employees it is 34.
- The median income of atrited employyes is lower (\~51K) than non attrited employees (\~64K).
- The median total business value generated by attrited employees (\~46K) is almost 1/6 of the non attrited employees (\~263K)
- The median experience for attrited employees is about 165 days and for non attrited employees it is 186 days.
- The attrition rate of male and female employees is similar (\~67-68%).
- The attrition rate across different education levels is also similar (from 66-69%).
- The attrition among those who received raise (7%) is far lower than those who did not receive raise (68%).
- The atrition is highest among those whose current designation is 1 (80%) followed by 2 (70%). The rate is around 50-54% for the remaining designation.
- The attrition rate is 70% for those who were not promoted and 54% for those who were promoted.
- The attrtion rate follows a clear declinig pattern with increasing quarterly rating : 1 (82%), 2 (40%), 3 (16%) and 4 (9%).
- The attrtion rate is highest among those whose quarterly rating decreased (81%), followed by those whose rating remained unchanged (74%) and finally those whose rating increased (23%).
- There is mopderate correlation between the Total Business Value and Experience features, which is somewhat expected, since the longer a person works, the more likely it is for him to generate positive business value and higher the sum of monthly business values.
- There is moderate correlation between Current Designation and Income which is expected since the higher the designation, the greater the income.
- Also, there is moderate correlation between the current designation and joining designation.
- Again, there is moderate correlation between Experience and Promotion (the longer a person has been working, the higher the chance of getting promoted.

<a id = "thirteen"></a>
<h1 id="basics" style="background:black; padding: 10px 0; border:#FFC300; border-width:5px; border-radius: 10px; border-style:solid; font-family:MV Boli; color:white; line-height: 10px;"> 
    <center> ◽ Discussion on Trade Off and Possible Recommendations ◽
    </center>
</h1>

#### 
#### The Trade-Off

In general while choosing a model, we might choose to look at precision and recall scores and choose while keeping the following trade-off on mind :
- If we prioritize precision, we are going to reduce our false positives. This may be useful if our targeted retention strategies prove to be expensive. We don't want to spend unnecessarily on somebody who is not even going to leave in the first place. Also, it might lead to uncomfortable situation for the employee themselves if they are put in a situation where it is assumed that they are going to be let go/ going to leave.

- If we prioritize recall, we are going to reduce our false negatives. This is useful since usually the cost of hiring a new person is higher than retaining an experienced person. So, by reducing false negatives, we would be able to better identify those who are actually going to leave and try to retain them by appropriate measures (competitive remuneration, engagement program, etc). 

### Recommendations 

- We saw that the percentage of employees who received a raise was only 1.8%, whereas 17% of the employees received a promotion. It appears that promotion does not guarantee an increase in compensation. Employees may not be motivated to continue working for an organization if they feel that they are undervalued in terms of financial compensation. This is further substantiated by the fact that attrition rate among promoted employees was 54%, which was a lot higher than the attrition rate of employees who received a raise (7%). So it is better if promotion also leads to increase in compensation.

- We saw that quarterly rating is an important indicator of whether the employee might leave voluntarily/involuntarily. A lot of employees have the lowest quarterly rating and the attrition among them is high. The organization might look into improvement programs for underperforming employees before they are let go, or before they resign. Also, improvement in rating can be rewarded to encourage better performance. 

- Salaries must be competitve to prevent losing employees to attrition and to competition and since almost 68% of employees attrited in a small span, it seems that correction of payscales is necessary for the organization since hiring new employees is costlier than retaining old ones.

- Specific employee engagement programs can also be introduced targeting the different categories of performers to help them achieve their goals as well as help them increase their contribution towards the business value. The hiring process can also be restructured since a lot of new hires seem to be leaving (median experience is lower for attrited employees).

### Thank You for Reading! Please add your suggestions/comments and upvote if you liked my work!